In [1]:
from pathlib import Path
from time import sleep 

import numpy as np

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator import VisualRobot
from manipylator import utils

# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
# path = Path('/workspace/robots/robot.urdf')
path = Path('/workspace/robots/robot-ee.urdf')
manny = VisualRobot(path)
print(manny.model)

[I 05/21/25 22:13:42.841 1960] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
[Genesis] [22:13:48] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [22:13:48] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [22:13:48] [INFO] ╰───────────────────────────────────────────────╯
/opt/conda/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
[Genesis] [22:13:49] [INFO] Running on [Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz] with backend gs.vulkan. Device memory: 15.25 GB.
RHI Error: Can not create Vulkan instance
[W 05/21/25 22:13:49.004 1960] [misc.py:adaptive_arch_select@758] Arch=[<Arch.v

ERobot: onshape, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │              ETS: parent to link               │
├──────┼──────────────────┼───────┼──────────────────┼────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                          │
│    1 │ carriage_1       │     0 │ base             │ SE3(0, 0, 0.037; -180°, -0°, 0°) ⊕ Rz(q0)      │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, 0, -0.055; 0°, 90°, -0°) ⊕ Rz(q1)  │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.35, 0, 0; -180°, -0°, 0°) ⊕ Rz(q2)       │
│    4 │ wrist_carriage_2 │     3 │ elbow            │ SE3(0.32, 0, -0.025; 180°, -0°, 0°) ⊕ Rz(q3)   │
│    5 │ wrist_carriage_3 │     4 │ wrist_carriage_2 │ SE3(0.04, 0, -0.0485; 0°, -90°, -0°) ⊕ Rz(q4)  

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space. Below we find the location of the end effector in when the all joint parameters are set to 0 degrees:

In [3]:
robot = manny.visualizer.robot
# robot

In [4]:
ee = robot.get_link('end_effector')
# ee

In [5]:
ee_initial_pos = ee.get_pos()
ee_initial_pos

tensor([-1.2850e-01, -1.7960e-17,  8.5050e-01])

In [6]:
ee_initial_quat = ee.get_quat()
ee_initial_quat

tensor([ 3.5732e-16, -7.0711e-01,  2.7072e-16,  7.0711e-01])

And we can verify that for the inverse is also true:

In [7]:
initial_qpos = robot.inverse_kinematics(
    link = ee,
    pos  = ee_initial_pos,
    quat = ee_initial_quat,
)
initial_qpos

[W 05/21/25 22:14:19.047 1960] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


tensor([0., 0., 0., 0., 0., 0.])

I.e. that when setting all the joints to 0 degrees the end effector end up at `ee_initial_pos` and that if the end effector is at `ee_initial_pos` all the joints are at 0 degress.

Similarly, for an arbitrary (reachable) location in 3D space:

In [8]:
qpos = robot.inverse_kinematics(
    link = ee,
    pos  = [-4.8500e-02, -4.7547e-02,  4.0200e-01],
    quat = ee_initial_quat,
)
qpos

tensor([-0.4818,  0.9675,  1.9853, -2.1086, -0.4815,  3.1416])

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [19]:


scale = 0.15
x_offset = -0.0885
y_offset = 0.48
height = 0.09

# -0.0885,  0.2954,  0.2595
shape = utils.parametric_circle_1(np.linspace(0, 2*np.pi, 20))
positions = scale * shape
positions[:,0] += x_offset
positions[:,1] += y_offset
positions[:,2] = height

positions[:3]

array([[0.0615    , 0.48      , 0.09      ],
       [0.05337259, 0.52870492, 0.09      ],
       [0.02987108, 0.57213191, 0.09      ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [20]:
# for x, y, z in positions:
#     manny.visualizer.scene.draw_debug_sphere([x,y,z])
manny.visualizer.scene.clear_debug_objects()
manny.visualizer.scene.draw_debug_spheres(positions)



Similarly, we can calculate the joint parameters of each position:

In [21]:
robot.set_dofs_position([0, 0.2, 2.35, 1, 1.57, 0])

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0885,  0.2954,  0.2595])

In [22]:
quat_pointing_down = [0, -1, 0, 0]

initial_pose = robot.inverse_kinematics(
        link = ee,
        pos  = positions[0],
        quat = quat_pointing_down,
        )

robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

In [14]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()

In [15]:
camera = manny.visualizer.camera

camera.start_recording()
# [camera.render() for i in range(10)]

# Reset to initial curve pose
robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

camera.render()

(array([[[35, 53, 71],
         [35, 53, 71],
         [35, 53, 71],
         ...,
         [38, 55, 74],
         [39, 57, 76],
         [41, 58, 77]],
 
        [[34, 52, 70],
         [34, 51, 70],
         [33, 51, 69],
         ...,
         [42, 59, 78],
         [42, 60, 78],
         [42, 60, 79]],
 
        [[26, 44, 62],
         [26, 44, 62],
         [26, 44, 62],
         ...,
         [38, 56, 75],
         [37, 54, 73],
         [35, 53, 71]],
 
        ...,
 
        [[21, 37, 53],
         [21, 37, 53],
         [21, 37, 53],
         ...,
         [22, 39, 55],
         [22, 39, 55],
         [22, 39, 55]],
 
        [[21, 37, 53],
         [21, 37, 53],
         [21, 37, 53],
         ...,
         [22, 39, 55],
         [22, 39, 55],
         [22, 39, 55]],
 
        [[21, 37, 53],
         [21, 37, 53],
         [21, 37, 53],
         ...,
         [22, 39, 55],
         [22, 39, 55],
         [22, 39, 55]]], dtype=uint8),
 None,
 None,
 None)

In [23]:

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = quat_pointing_down,
        ) for pos in positions]
qs[:5]

[tensor([ 0.3113, -0.5833,  1.9051,  0.9177,  1.5708, -1.8822]),
 tensor([ 0.2679, -0.6746,  1.7224,  0.8263,  1.5708, -1.8387]),
 tensor([ 0.2075, -0.7597,  1.5511,  0.7400,  1.5708, -1.7783]),
 tensor([ 0.1360, -0.8325,  1.4037,  0.6654,  1.5708, -1.7068]),
 tensor([ 0.0590, -0.8846,  1.2979,  0.6117,  1.5708, -1.6298])]

And to visualize the poses Manny would take, we can animate the path:

In [24]:
for q in qs:
    robot.set_dofs_position(q)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    # camera.render()
    sleep(0.1)

camera.stop_recording(save_to_filename='video.mp4', fps=20)

NameError: name 'camera' is not defined

In [18]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()
